In [94]:
import numpy as np
import pandas as pd
import pickle
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Load pickled data
with open('events.p', 'rb') as f:
    events = pickle.load(f)

with open('teams.p', 'rb') as f:
    teams = pickle.load(f)
    
with open('matches.p', 'rb') as f:
    matches = pickle.load(f)
    
with open('players.p', 'rb') as f:
    players = pickle.load(f)

In [3]:
match_dicts = { match['match_id']: match for match in matches }

In [4]:
# Function to include weather data to each event
def add_weather(event):
    weather = match_dicts[event['match_id']]
    event['temp_c'] = weather['temp_c']
    event['wind_kmph'] = weather['wind_kmph']
    event['precip_mm'] = weather['precip_mm']
    return event

In [5]:
events = list(map(add_weather, events))

In [6]:
df = pd.DataFrame(events)

In [7]:
# Create pass specific DataFrame
pass_df = df.loc[df['event'] == 'Pass'][['player', 'tags', 'subevent', 'precip_mm']].copy()

In [8]:
# Function to code action success
def acc_or_not(tags):
    if 'Accurate' in tags:
        return 1
    else:
        return 0

In [9]:
pass_df['tags'] = pass_df['tags'].map(acc_or_not)

In [126]:
# Bin precipitation
x = pass_df['precip_mm']

conditions = [
    x == 0,
    (x > 0) & (x <= 0.5),
    (x > 0.5) & (x <= 1),
    (x > 1) & (x <= 2),
    x > 2
]

choices = [
    'no',
    'very_light',
    'light',
    'medium',
    'heavy'
]

pass_df['precipitation'] = np.select(conditions, choices)

In [127]:
pdf = pd.DataFrame(
    pass_df.loc[
        (pass_df['subevent'] != 'Hand pass') & (pass_df['subevent'] != 'Launch')
    ].groupby(
        ['precipitation']
    )['subevent'].value_counts(normalize=True)
).unstack()

In [128]:
pdf.columns = [ col[1] for col in pdf.columns.tolist() ]

In [129]:
pdf = pdf.reindex(['no', 'very_light', 'light', 'medium', 'heavy'])

In [130]:
fig = go.Figure(data=[
    go.Bar(name=p_type, x=pdf.index, y=pdf[p_type]) for p_type in pdf.columns
])
fig.update_layout(
    title='Rate of Pass Type by Level of Precipitation',
    xaxis_title='Amount of Rain',
    yaxis_title='Rate of Pass Type'
)
fig.show()